In [1]:
import tensorflow as tf
import math
from tensorflow.examples.tutorials.mnist import input_data as mnist_data
#print("Tensorflow version " + tf.__version__)
#tf.set_random_seed(0)

In [2]:
config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )
sess = tf.Session(config=config)

In [3]:
def get_score(n_iter = 10, layer_L = 200, layer_M = 100, layer_N = 60, layer_O = 30, max_learning_rate = 0.003, 
              min_learning_rate = 0.0001 ,decay_speed = 2000.0, train_batch_size = 100, drop_out_rate = 0.75):
    
    # layer_L > 1K (10x-100x)
    # layer_M > 500
    # layer_N > 100
    # layer_0 > 50

    ##
    ## data prep
    ##
    mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=False, validation_size=0)

    # input X: 28x28 grayscale images, the first dimension (None) will index the images in the mini-batch
    X = tf.placeholder(tf.float32, [None, 28, 28, 1])
    # correct answers will go here
    Y_ = tf.placeholder(tf.float32, [None, 10])
    # variable learning rate
    lr = tf.placeholder(tf.float32)
    # Probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
    pkeep = tf.placeholder(tf.float32)
    
    ##
    ## architecture setup
    ##
    L = layer_L
    M = layer_M
    N = layer_N
    O = layer_O
    # Weights initialised with small random values between -0.2 and +0.2
    # When using RELUs, make sure biases are initialised with small *positive* values for example 0.1 = tf.ones([K])/10
    W1 = tf.Variable(tf.truncated_normal([784, L], stddev=0.1))  # 784 = 28 * 28
    B1 = tf.Variable(tf.ones([L])/10)
    W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
    B2 = tf.Variable(tf.ones([M])/10)
    W3 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
    B3 = tf.Variable(tf.ones([N])/10)
    W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
    B4 = tf.Variable(tf.ones([O])/10)
    W5 = tf.Variable(tf.truncated_normal([O, 10], stddev=0.1))
    B5 = tf.Variable(tf.zeros([10]))


    # The model, with dropout at each layer
    XX = tf.reshape(X, [-1, 28*28])

    Y1 = tf.nn.relu(tf.matmul(XX, W1) + B1)
    Y1d = tf.nn.dropout(Y1, pkeep)

    Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + B2)
    Y2d = tf.nn.dropout(Y2, pkeep)

    Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + B3)
    Y3d = tf.nn.dropout(Y3, pkeep)

    Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + B4)
    Y4d = tf.nn.dropout(Y4, pkeep)

    Ylogits = tf.matmul(Y4d, W5) + B5
    Y = tf.nn.softmax(Ylogits)


    # cross-entropy loss function (= -sum(Y_i * log(Yi)) ), normalised for batches of 100  images
    # TensorFlow provides the softmax_cross_entropy_with_logits function to avoid numerical stability
    # problems with log(0) which is NaN
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)
    cross_entropy = tf.reduce_mean(cross_entropy)*100

    # accuracy of the trained model, between 0 (worst) and 1 (best)
    correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # training step, the learning rate is a placeholder
    train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

    # init
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)

    def training_step(i):

        # training on batches of 100 images with 100 labels
        batch_X, batch_Y = mnist.train.next_batch(train_batch_size)

        # learning rate decay
        #max_learning_rate = 0.003
        #min_learning_rate = 0.0001
        #decay_speed = 2000.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations
        learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)

        # the backpropagation training step
        sess.run(train_step, {X: batch_X, Y_: batch_Y, pkeep: drop_out_rate, lr: learning_rate})

    for i in range(n_iter):
        training_step(i)
    a, c= sess.run([accuracy, cross_entropy], {X: mnist.test.images, Y_: mnist.test.labels, pkeep: 1.0})
    
    return a

In [7]:
%%time
get_score(n_iter=1000)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
CPU times: user 5.64 s, sys: 2.37 s, total: 8.01 s
Wall time: 6.32 s


0.96220011